In [1]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt
%pip install -q roboflow

import torch
import os
import shutil
from IPython.display import Image, clear_output


Cloning into 'yolov5'...
remote: Enumerating objects: 16559, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 16559 (delta 18), reused 29 (delta 13), pack-reused 16522
Receiving objects: 100% (16559/16559), 15.08 MiB | 9.80 MiB/s, done.
Resolving deltas: 100% (11375/11375), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.8/750.8 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.3.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install roboflow

os.environ["DATASET_DIRECTORY"] = "/content/datasets"
yolov5_directory = '/content/yolov5'
os.environ["YOLOV5_DIRECTORY"] = yolov5_directory

from roboflow import Roboflow
rf = Roboflow(api_key='TiExgP2alzxgNpQViv9O')
project = rf.workspace().project('maskdetector-wujbz')
dataset = project.version(1).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to /content/datasets/MaskDetector-1 in yolov5pytorch:: 100%|██████████| 3417/3417 [00:00<00:00, 6048.56it/s]


In [4]:
yolov5_model = 'n'
config = 'yolov5' + yolov5_model + '.yaml'
startWeights = 'yolov5' + yolov5_model + '.pt'

os.environ["YOLO_CONFIG"] = config
os.environ["INIT_WEIGHTS"] = startWeights

In [5]:
!python train.py --img 416 --batch-size -1 --epochs 300 --data {dataset.location}/data.yaml --cfg {config} --cache

2024-04-18 23:05:13.061918: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-18 23:05:13.061977: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-18 23:05:13.063388: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=yolov5n.yaml, data=/content/datasets/MaskDetector-1/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=-1, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device

In [6]:
def getLastCreatedExp():
  """
    Returns name of last created exp folder (located in /runs/train/...)
  """
  contents = os.listdir(f'{yolov5_directory}/runs/train')
  contentsWithoutZIP = []
  for f in contents:
    if '.zip' not in f and 'exp' in f:
      contentsWithoutZIP.append(f)
  contentsWithoutZIP.sort()
  return contentsWithoutZIP[-1]

last_exp = getLastCreatedExp()



In [7]:
# evaluates trained model on test set
!python val.py --data {dataset.location}/data.yaml --batch-size 8 --weights {yolov5_directory}/runs/train/{last_exp}/weights/last.pt

val: data=/content/datasets/MaskDetector-1/data.yaml, weights=['/content/yolov5/runs/train/exp/weights/last.pt'], batch_size=8, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-304-g22361691 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5n summary: 157 layers, 1763224 parameters, 0 gradients, 4.1 GFLOPs
val: Scanning /content/datasets/MaskDetector-1/valid/labels.cache... 171 images, 0 backgrounds, 0 corrupt: 100% 171/171 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances        

Weights and train results are located in `runs/train/exp*` whiel test results are saved in `runs/val/exp*`.

